#  Molecular Property Prediction with Quantum Transformer

**Quantum Transformer Tutorial 02**

Apply the Quantum Transformer to predict molecular properties from SMILES strings.

In [ ]:
import torch
from quantum_transformers.molecular import (
    QuantumTransformerForMolecules,
    MolecularEnergyPredictor,
    SMILESTokenizer,
    MolecularModelConfig,
)

## 1. Tokenizing Molecules

In [ ]:
tokenizer = SMILESTokenizer()

molecules = [
    "CCO",      # Ethanol
    "CC(=O)O",  # Acetic acid
    "c1ccccc1", # Benzene
]

for mol in molecules:
    tokens = tokenizer.tokenize(mol)
    print(f"{mol:12s} → {tokens}")

## 2. Create Molecular Quantum Transformer

In [ ]:
config = MolecularModelConfig(
    vocab_size=tokenizer.vocab_size,
    n_qubits=2,
    n_heads=2,
    n_layers=2,
    d_model=16,
    task="energy",
)

model = QuantumTransformerForMolecules(config)
print(f"Model created with {sum(p.numel() for p in model.parameters())} parameters")

In [ ]:
# Predict energies
tokens = tokenizer(molecules, max_length=16, return_tensors="pt")
energies = model(tokens)

for mol, energy in zip(molecules, energies):
    print(f"{mol:12s} → Energy: {energy.item():.4f}")

## 3. Training the Model

In [ ]:
from quantum_transformers.optimization import QuantumAdamOptimizer

# Synthetic training data
train_smiles = ["C", "CC", "CCC", "CCO", "CCCO", "c1ccccc1"]
train_energies = torch.tensor([[-1.0], [-2.0], [-3.0], [-2.5], [-3.5], [-5.0]])

train_tokens = tokenizer(train_smiles, max_length=16, return_tensors="pt")

optimizer = QuantumAdamOptimizer(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training loop
for epoch in range(20):
    optimizer.zero_grad()
    pred = model(train_tokens)
    loss = criterion(pred, train_energies)
    loss.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

## Summary

- SMILESTokenizer converts molecules to tokens
- QuantumTransformerForMolecules predicts properties
- Training uses quantum-aware optimization

**Next**: [03_quantum_attention_analysis.ipynb](03_quantum_attention_analysis.ipynb)